# 0 - Imports and Packages needed

In [33]:
import pymysql
import pandas as pd
import numpy as np

# 1 - Data Preparation and Cleaning

In [19]:
# Reading data using pandas, specifying sheet names --> one sheet for table
dataset_path = './dataset/SQL Test Data.xlsx'

account = pd.read_excel(dataset_path, sheet_name="Account")
customer = pd.read_excel(dataset_path, sheet_name="Customer")
betting = pd.read_excel(dataset_path, sheet_name="Betting")
product = pd.read_excel(dataset_path, sheet_name="Product")
student_school = pd.read_excel(dataset_path, sheet_name="Student_School", header = 1)

In [21]:
# Splitting the student school table correctly
student = student_school.iloc[1:,:4]
school = student_school.iloc[:,-3:]


In [23]:
# Dropping unnecessary columns
betting = betting.drop(columns = ['Unnamed: 9','Unnamed: 10','Unnamed: 11','Unnamed: 15'])
betting = pd.concat((betting.iloc[:,:19:], betting.iloc[:,20]), axis = 1) # concatenating last column and the other to remove the column nan
betting

,AccountNo,BetDate,ClassId,CategoryId,Source,BetCount,Bet_Amt,Win_Amt,Product,AccountNo.1,Bet_Amt.1,Product.1,AccountNo.2,Vegas,Sportsbook,Games,Casino,Poker,Bingo,Adjustments
0,01196ZZ,2012-11-24,22512_XFER,22512,I,189,29364.0,28947.00,Vegas,01196ZZ,29364.0,Vegas,01196ZZ,89860.7,42758.00,0.00,1262.0,0.0,0.0,0.00
1,00357DG,2012-11-27,22512_XFER,22512,I,1415,19835.0,18639.15,Vegas,00357DG,19835.0,Vegas,00357DG,191804.9,100.00,16899.85,20.0,0.0,49.8,295.21
2,00357DG,2012-11-24,22512_XFER,22512,I,2026,19280.8,17904.50,Vegas,00357DG,19280.8,Vegas,01284UW,0.0,152613.05,0.00,0.0,0.0,0.0,0.00
3,01196ZZ,2012-12-19,22512_XFER,22512,I,151,17301.0,16956.00,Vegas,01196ZZ,17301.0,Vegas,01148BP,7030.0,290957.71,0.00,0.0,0.0,0.0,0.00
4,01284UW,2012-10-21,46,5,I,25,16850.0,3319.11,Sportsbook,01284UW,16850.0,Sportsbook,00497XG,0.0,0.00,25784.57,0.0,0.0,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1015,01196ZZ,2012-12-19,22522_SALE,22522,I,0,0.0,20.00,Vegas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1016,01284UW,2012-10-15,274,5,I,0,0.0,10003.50,Sportsbook,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1017,01284UW,2012-11-05,424,17,I,1,0.0,0.00,Sportsbook,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1018,01284UW,2012-12-21,46,5,I,0,0.0,103.71,Sportsbook,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
# Additional cleaning steps that were causing errors mainly the presence of duplicates and numpy data types

account = account.drop_duplicates().replace({np.nan: None})
customer = customer.drop_duplicates().replace({np.nan: None})
betting = betting.drop_duplicates().replace({np.nan: None}).dropna()

product = product.drop_duplicates().replace({np.nan: None})
student = student.drop_duplicates().replace({np.nan: None})
school = school.drop_duplicates().replace({np.nan: None}).dropna()


# 2 - Connect to Database

In [53]:
#Authentication
cnx = pymysql.connect(user='root', password='root', host='localhost')
if cnx.open:
    print("Connection open")
else:
    print("Connection is not successfully open")


Connection open


In [55]:
# Creating cursor object to interact with database
cursor = cnx.cursor()

In [59]:
## UNCOMMENT TO DESTROY DATABASE
query = "DROP DATABASE IF EXISTS ironhack_gambling"
cursor.execute(query)

0

In [61]:
## Query to create database write than execute using cursor
query = ("""CREATE DATABASE IF NOT EXISTS ironhack_gambling""")
cursor.execute(query)

1

In [67]:
## Queries to create our tables, specifying table name and Database at the beggining,
## next we add columns along with some of their data types.

query = ("""CREATE TABLE IF NOT EXISTS
ironhack_gambling.account(
AccountNo VARCHAR(52) PRIMARY KEY,
CustId INT NOT NULL,
AccountLocation VARCHAR(32) NOT NULL,
CurrencyCode VARCHAR(32) NOT NULL,
DailyDepositLimit INT NOT NULL,
StakeScale FLOAT NOT NULL,
SourceProd VARCHAR(32))""")
cursor.execute(query)

query = ("""CREATE TABLE IF NOT EXISTS
ironhack_gambling.customer(
CustID INT PRIMARY KEY,
AccountLocation VARCHAR(32) NOT NULL,
Title VARCHAR(32) NOT NULL,
FirstName VARCHAR(64) NOT NULL,
LastName VARCHAR(64) NOT NULL,
CreateDate VARCHAR(32),
CountryCode VARCHAR(32),
Language VARCHAR(32),
Status VARCHAR(32),
DateOfBirth VARCHAR(32),
Contact VARCHAR(32),
CustomerGroup VARCHAR(32))""")
cursor.execute(query)

query = ("""CREATE TABLE IF NOT EXISTS
ironhack_gambling.betting(
AccountNo VARCHAR(52) NOT NULL,
BetDate TIMESTAMP NOT NULL ,
ClassId VARCHAR(32) NOT NULL,
CategoryId VARCHAR(64) NOT NULL,
Source VARCHAR(32) NOT NULL,
BetCount INT NOT NULL,
Bet_Amt INT NOT NULL,
Win_Amt FLOAT NOT NULL,
Product VARCHAR(32) NOT NULL,
AccountNo_1 VARCHAR(32) NOT NULL,
Bet_Amt_1 FLOAT NOT NULL,
Product_1 VARCHAR(32),
AccountNo_2 VARCHAR(32),
Vegas FLOAT ,
Sportsbook FLOAT,
Games FLOAT,
Casino FLOAT,
Poker FLOAT,
Bingo FLOAT,
Adjustments FLOAT)""")
cursor.execute(query)

query = ("""CREATE TABLE IF NOT EXISTS
ironhack_gambling.product(
CLASSID VARCHAR(32),
CATEGORYID INT NOT NULL,
Product VARCHAR(32) NOT NULL,
sub_product VARCHAR(32) NOT NULL,
description VARCHAR(64),
bet_or_play INT NOT NULL)""")
cursor.execute(query)

query = ("""CREATE TABLE IF NOT EXISTS
ironhack_gambling.school(
school_id_1 VARCHAR(32) PRIMARY KEY,
school_name VARCHAR(32) NOT NULL,
city_1  VARCHAR(32) NOT NULL
)""")
cursor.execute(query)

query = ("""CREATE TABLE IF NOT EXISTS
ironhack_gambling.student(
student_id VARCHAR(32) PRIMARY KEY,
student_name  VARCHAR(32) NOT NULL,
city VARCHAR(32) NOT NULL,
school_id VARCHAR(32),
GPA VARCHAR(32) NOT NULL DEFAULT 0
)""")
cursor.execute(query)

0

In [69]:
# Changes are not final and approved until you commit
# Once you run this cell you should see your new database
cnx.commit()

In [73]:
# Finally we add the rows and entries to our tables
data = [account, customer, betting, product, student, school] # list of tables
data_title = ['account', 'customer', 'betting','product','student', 'school'] # names of tables

## We will do 2 for loops 1 to loop over the tables
#### The inner one for populating rows using the insert into statement


for j in range(len(data)):    ## Loop over data
    table = data[j]            ## get the table
    #print(data_title[j])      ## ----> Using print statements to debug
    columns = table.columns    ## Needed to create our query as the insert into takes in the column names and the values as a parameter   
    parameter = ", ".join(list(columns)) ## Creating the string for the list of columns
    parameter = parameter.replace(".","_")  ## Replacing the  . with - in column names that have a . so that we don't have a problem with SQL statements
    param_2 = ("%s, " * len(columns))[:-2]  ## Creating the placeholder for the value
    
    query = f"INSERT INTO ironhack_gambling.{data_title[j]}({parameter}) VALUES ({param_2})"
    # Creating the query for the table
    
    for i in range(len(data[j])):
        row = list(table.iloc[i])        ## Getting the parameters to be passed which are the values in the row itself
        #print(row)                      ## ----> Using print statements to debug
        cursor.execute(query, (row))     ## Executing the query and passing the row as argument so that for each table, 
                                         ##   each rown should be
                                        

In [75]:
## Finally don't forget to commit
cnx.commit()

In [105]:
cursor.execute("""SELECT * FROM ironhack_gambling.account""") # Check if Account Table was populated
account_tbl_rows = cursor.fetchall()

column_names = [desc[0] for desc in cursor.description] # Get Columns Name
print(column_names)

for row in account_tbl_rows: # Get Table Rows
    print(row)

['AccountNo', 'CustId', 'AccountLocation', 'CurrencyCode', 'DailyDepositLimit', 'StakeScale', 'SourceProd']
('00357DG   ', 3531845, 'GIB', 'GBP', 0, 1.0, 'GM')
('00497XG   ', 4188499, 'GIB', 'GBP', 0, 1.0, 'SB')
('00692VS   ', 4704925, 'GIB', 'USD', 0, 2.0, 'SB')
('00775SM   ', 2815836, 'GIB', 'USD', 0, 1.0, 'SB')
('00C017    ', 889782, 'GIB', 'GBP', 1500, 0.41, 'XX')
('00J381    ', 1191874, 'GIB', 'GBP', 500, 8.0, 'XX')
('01148BP   ', 1569944, 'GIB', 'GBP', 0, 8.0, 'XX')
('01152SJ   ', 1965214, 'GIB', 'USD', 0, 1.0, 'PO')
('01196ZZ   ', 3042166, 'GIB', 'EUR', 0, 8.0, 'SB')
('01284UW   ', 5694730, 'GIB', 'GBP', 0, 1.0, 'SB')
